## Timetabling Program for SMUS Middle School
#### Copyright © 2020 Hoshino Math Services

In [ ]:
#import sys
#!{sys.executable} -m pip install ortools 

# import modules
import time
import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp

In [ ]:
# Import all the Data for Grades 6-8
InputFile = pd.ExcelFile("Middle School Input.xlsx") 
Grade6Matrix = pd.read_excel(InputFile, 'Grade6') 
Grade6Info = Grade6Matrix.values.tolist()
Grade7Matrix = pd.read_excel(InputFile, 'Grade7') 
Grade7Info = Grade7Matrix.values.tolist()
Grade8Matrix = pd.read_excel(InputFile, 'Grade8') 
Grade8Info = Grade8Matrix.values.tolist()


# Generate the set of Courses and Teachers at the Middle School
AllCourses = []
AllTeachers = []
for j in [1,5,9,13]:
    for i in range(2, len(Grade6Info)-1):
        Course = Grade6Info[i][j]
        Teacher = Grade6Info[i][j+2]
        if not Course in AllCourses:
            AllCourses.append(Course)
        if not Teacher in AllTeachers and not pd.isna(Teacher):
            AllTeachers.append(Teacher)
    for i in range(2, len(Grade7Info)-1):
        Course = Grade7Info[i][j]
        Teacher = Grade7Info[i][j+2]
        if not Course in AllCourses:
            AllCourses.append(Course)
        if not Teacher in AllTeachers and not pd.isna(Teacher):
            AllTeachers.append(Teacher)
    for i in range(2, len(Grade8Info)-1):
        Course = Grade8Info[i][j]
        Teacher = Grade8Info[i][j+2]
        if not Course in AllCourses:
            AllCourses.append(Course)
        if not Teacher in AllTeachers and not pd.isna(Teacher):
            AllTeachers.append(Teacher)
            
            
# For each (Grade, Section, Course) triplet, determine the Teacher of that class.

GSCTeacher = [ [['Not Applicable' for c in range(40)] for s in range(5)] for g in range(10)]
for j in [1,5,9,13]:
    s = int((j+3)/4)
    for i in range(2, len(Grade6Info)-1):
        Course = Grade6Info[i][j]
        Teacher = Grade6Info[i][j+2]
        c = AllCourses.index(Course)
        if not pd.isna(Teacher):
            GSCTeacher[6][s][c] = Teacher
    for i in range(2, len(Grade7Info)-1):
        Course = Grade7Info[i][j]
        Teacher = Grade7Info[i][j+2]
        c = AllCourses.index(Course)
        if not pd.isna(Teacher):
            GSCTeacher[7][s][c] = Teacher
    for i in range(2, len(Grade8Info)-1):
        Course = Grade8Info[i][j]
        Teacher = Grade8Info[i][j+2]
        c = AllCourses.index(Course)
        if not pd.isna(Teacher):
            GSCTeacher[8][s][c] = Teacher

In [ ]:
# Optimize assignment of courses to blocks

solver = pywraplp.Solver('SMUS Middle School', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

start_time = time.time()


# Define our Grades, Sections, Days, Periods, Courses
Grades = [6,7,8]
Sections = [1,2,3,4]
Days = [1,2,3,4,5]
Periods = [1,2,3,4,5,6,7]
Courses = range(len(AllCourses))


# Define our five-dimensional Boolean Variables
x = {}
for g in Grades:
    for s in Sections:
        for d in Days:
            for p in Periods:
                for c in Courses:
                    x[g,s,d,p,c] = solver.IntVar(0,1, 'x[%d,%d,%d,%d,%d]' % (g,s,d,p,c))

        
# CONSTRAINT 1: For each grade and each section, there is exactly one course each period.
for g in Grades:
    for s in Sections:
        for d in Days:
            for p in Periods:
                solver.Add( sum(x[g,s,d,p,c] for c in Courses) == 1)
        
                        
# CONSTRAINT 2: Each course must be offered a set number of lessons in each week
       
for s in Sections:
    for i in range(2, len(Grade6Info)-1):
        CourseName = Grade6Info[i][4*s-3]
        Lessons = Grade6Info[i][4*s-2]
        c = AllCourses.index(CourseName)
        solver.Add(sum(x[6,s,d,p,c] for d in Days for p in Periods) == Lessons)
    for i in range(2, len(Grade7Info)-1):
        CourseName = Grade7Info[i][4*s-3]
        Lessons = Grade7Info[i][4*s-2]
        c = AllCourses.index(CourseName)
        solver.Add(sum(x[7,s,d,p,c] for d in Days for p in Periods) == Lessons)
    for i in range(2, len(Grade8Info)-1):
        CourseName = Grade8Info[i][4*s-3]
        Lessons = Grade8Info[i][4*s-2]
        c = AllCourses.index(CourseName)
        solver.Add(sum(x[8,s,d,p,c] for d in Days for p in Periods) == Lessons)

        
# CONSTRAINT 3: Certain courses must be offered in certain blocks
# Chapel on Tuesday Period 1, Assembly on Friday Period 1, XPLO on Friday Period 7

for g in Grades:
    for s in Sections:
        for c in [AllCourses.index('Chapel')]:
            solver.Add( x[g,s,2,1,c] == 1)
        for c in [AllCourses.index('Assembly')]:
            solver.Add( x[g,s,5,1,c] == 1)
        for c in [AllCourses.index('XPLO')]:
            solver.Add( x[g,s,5,7,c] == 1)


# CONSTRAINT 4: No teacher can teach two different classes simultaneously
# CONSTRAINT 5: No teacher teaches more than FOUR lessons on any given day

for Teacher in AllTeachers:
    Tset = []
    for g in Grades:
        for s in Sections:
            for c in Courses:    
                if GSCTeacher[g][s][c] == Teacher:
                    GradeSectionCourse = [g, s, c]
                    Tset.append(GradeSectionCourse)
    
    for d in Days:
        for p in Periods:
            solver.Add(sum(x[Tset[z][0],Tset[z][1],d,p,Tset[z][2]] 
                           for z in range(len(Tset))) <= 1)

    for d in Days:
        solver.Add(sum(x[Tset[z][0],Tset[z][1],d,p,Tset[z][2]]
                       for z in range(len(Tset)) for p in Periods) <= 4)
       
                  
# CONSTRAINT 6: All Grade 6/7/8 students must have at least one Humanities class each day.
c = AllCourses.index('Humanities')
for g in [6,7,8]:
    for s in Sections:
        for d in Days:
            solver.Add( sum(x[g,s,d,p,c] for p in Periods) >= 1)

            
# CONSTRAINT 7: All Grade 6/7 students must have at least one Math/Sci class each day.
c = AllCourses.index('Math/Science')
for g in [6,7]:
    for s in Sections:
        for d in Days:
            solver.Add( sum(x[g,s,d,p,c] for p in Periods) >= 1)
                      
        
# CONSTRAINT 8: A student can't have two lessons in the same class in non-consecutive periods.

for p1 in Periods:
    for p2 in Periods:
        if p2-p1>1:
            for g in Grades:
                for s in Sections:
                    for d in Days:
                        for c in Courses:
                            solver.Add(x[g,s,d,p1,c] + x[g,s,d,p2,c] <=1)

                            
# SET PREFERENCE COEFFICIENTS
w = [[[[[1 for c in range(40)] for p in range(10)] for d in range(10)] for s in range(10)] for g in range(10)] 

for g in [6,7]:
    for s in Sections:
        for d in Days:
            for c in [AllCourses.index('Humanities'), AllCourses.index('Math/Science')]:
                w[g][s][d][1][c]=3
                w[g][s][d][2][c]=3
                w[g][s][d][3][c]=2
                w[g][s][d][4][c]=2
                

# RUN OPTIMIZATION

solver.Maximize(solver.Sum(w[g][s][d][p][c]*x[g,s,d,p,c] for g in Grades for s in Sections for d in Days
                           for p in Periods for c in Courses))
sol = solver.Solve()
print("")
print('Optimization Complete with Total Happiness Score of', round(solver.Objective().Value()))

# compute runtime
solving_time = time.time() - start_time

print('The code ran in', round(solving_time,1), 'seconds')

In [ ]:
Timetable = [ [ "-" for d in range(6)] for z in range(84)]
CohortNames = ["-", "A", "B", "C", "D"]

for z in range(84):
    Timetable[z][0] = "P" + str(1+int(z/12))

for d in Days:
    for p in Periods:
        for g in Grades:
            for s in Sections:
                for c in Courses:
                    if x[g,s,d,p,c].solution_value()==1:
                        CourseID = str(g) + CohortNames[s] + "-" + AllCourses[c]
                        TeacherID = " (" + GSCTeacher[g][s][c] + ")"
                        RowIndex = 12*(p-1)+4*(g-6)+(s-1)                        
                        Timetable[RowIndex][d] = CourseID + TeacherID

                        
                        
OurColumns = ["Period", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
FinalMatrix = pd.DataFrame(Timetable, columns=OurColumns)
FinalMatrix.to_excel("Optimal SMUS Middle School Timetable.xlsx", index = False)